# Теория к лабораторной работе 6. Генераторы. Декораторы. Модули

## Генераторы

При работе с определенными последовательностями, можно обработать их чуть иначе. Мы не будем сразу создавать всю коллекцию, а вместо этого напишем правило, по которому она должна генерироваться. Таким образом, вместо хранения всех значений мы будем создавать каждое следующее поодиночке, придерживаясь написанного правила. Чтобы было более наглядно, напишем код для получения элементов последовательности Фибоначчи:


In [1]:
def fibonacci():
    current = 0
    next = 1
    while True:
        yield current  # Здесь мы как бы говорим питону вернуть значение и сделать паузу
        current, next = next, current + next


fibonacci_sequence = fibonacci()
print(type(fibonacci_sequence))

for _ in range(10):
    print(next(fibonacci_sequence))

<class 'generator'>
0
1
1
2
3
5
8
13
21
34


Здесь мы не создаем всю последовательность, а просто описываем правило, по которому эта последовательность будет создаваться. Написанная функция является генератором.

> Генераторы — это объекты, которые лениво создают последовательность значений по одному, не храня всю коллекцию в памяти.

В функциях-генераторах вместо обычного `return` есть оператор `yield`. Он тоже возвращает значение, но помимо этого еще и замораживает состояние функции. При помощи функции `next()` мы возвращаем значение и размораживаем наш генератор. Код выполнится до того как снова наткнется на `yield`. Поэтому несмотря на то, что в функции есть бесконечный цикл, программа будет завершаться корректно, если мы будем вызывать `next()` ограниченное количество раз.

С функцией `next()` мы познакомились, когда изучали итераторы, почему же она применяется и здесь? На самом деле все генераторы являются также и итераторами, то есть для них реализованы методы `__iter__()` и `__next__()`. Давайте в этом убедимся, и попробуем вызвать генератор большее количество раз, чем предусмотрено в его реализации:


In [2]:
import random


def random_generator():
    for _ in range(3):  # Сгенерирует число только 3 раза
        yield random.randint(1, 100)


generator_to_break = random_generator()
print(next(generator_to_break))
print(next(generator_to_break))
print(next(generator_to_break))
print(next(generator_to_break))

69
97
80


StopIteration: 

Возникла ошибка `StopIteration`! То есть все так же как и у итератора. Еще мы могли проходить по самим элементам итератора при помощи цикла `for`. Этот цикл как раз ожидает `StopIteration`, поэтому его можно применить и к генераторам:


In [3]:
import random


def random_generator():
    for _ in range(3):
        yield random.randint(1, 100)


for number in random_generator():
    print(number)

98
92
77


Существенная разница состоит в том, что в обычном итераторе мы работаем с уже готовой последовательностью, а в генераторе мы создаем каждый следующий элемент «на лету», что экономит память.


## Декораторы

Декоратор — это способ модифицировать поведение функции без изменения её исходного кода. Технически декоратор — это функция, которая берёт другую функцию и возвращает новую функцию с изменённым/дополненным поведением.

Они бывают полезны, когда нужно расширить функционал (например, сделать красивую обводку из символов для функции, которая печатает строку) или скрыть за ним громоздкую реализацию фреймворка, предоставляя удобный интерфейс передачи параметров (например, фреймворк **FastAPI**).

Чтобы понять, как работают декораторы, сначала рассмотрим одно интересное свойство, на котором они построены — замыкание.

> Замыкание — это функция, которая запоминает значения из окружающей (внешней) области видимости, даже если эта область уже завершила своё выполнение.

Дело в том, что имена функций — тоже ссылки на объекты функции, поэтому если вложить одну функцию в другую и использовать переменные внешней функции во внутренней, то объекты, на которые они ссылаются, не будут удалены сборщиком мусора, и могут быть использованы, несмотря на то что внешняя функция завершилась:


In [4]:
def multiply_by_n(n: int):
    def multiplier(x: int) -> int:
        return n * x

    return multiplier


double = multiply_by_n(2)  # `double` будет ссылаться на объект функции `multiplier`
print(f"I'm not double! I am {double.__name__}!")
print(double(1024))  # В `double` `n` в `multiplier` ссылается на 2

I'm not double! I am multiplier!
2048


Во внешнюю функцию можно передавать не только обычные параметры, но и другие функции. Тогда такая конструкция станет декоратором. Давайте напишем декоратор, который будет строить коробку вокруг текста:

In [5]:
def beautiful_box(function):
    def wrapper(*args, **kwargs) -> str:
        text = function(*args, **kwargs)
        lines = text.split("\n")
        max_len = len(max(lines, key=len))
        border = f"+{'':->{max_len+2}}+"
        content = ""
        for line in lines:
            content += f"| {line:<{max_len}} |\n"
        return f"{border}\n{content}{border}"

    return wrapper


def get_some_text(text):
    return text


get_some_text = beautiful_box(get_some_text)  # Оборачиваем функцию в декоратор
print(get_some_text("Hello\nwoooooorld\nof\nmine"))

+------------+
| Hello      |
| woooooorld |
| of         |
| mine       |
+------------+


Чтобы каждый раз не прописывать конструкцию вида `function = decorator(function)`, разработчики добавили синтаксический сахар — оператор `@`. Его нужно поставить перед названием декорирующей функции и написать перед декорируемой функцией:

In [6]:
def beautiful_box(function):
    def wrapper(*args, **kwargs) -> str:
        text = function(*args, **kwargs)
        lines = text.split("\n")
        max_len = len(max(lines, key=len))
        border = f"+{'':->{max_len+2}}+"
        content = ""
        for line in lines:
            content += f"| {line:<{max_len}} |\n"
        return f"{border}\n{content}{border}"

    return wrapper


@beautiful_box  # Теперь будет красивый вывод
def get_some_text(text):
    return text


print(get_some_text("Hello\nwoooooorld\nof\nmine!"))

+------------+
| Hello      |
| woooooorld |
| of         |
| mine!      |
+------------+


Но что если мы хотим поменять оформление коробки на что-то другое, например, тильду? Было бы здорово, если бы можно было передать стиль оформления как параметр. Такое вполне реально сделать! Нужно лишь добавить наш декоратор в еще одну внешнюю функцию, которая будет принимать символ оформления коробки:

In [7]:
def beautiful_box(width_decor_symbol: str = "-", height_decor_symbol: str = "|"):
    def decorator(function):
        def wrapper(*args, **kwargs) -> str:
            text = function(*args, **kwargs)
            lines = text.split("\n")
            max_len = len(max(lines, key=len))
            border = f"+{'':{width_decor_symbol}>{max_len+2}}+"
            content = ""
            for line in lines:
                content += (
                    f"{height_decor_symbol} {line:<{max_len}} {height_decor_symbol}\n"
                )
            return f"{border}\n{content}{border}"

        return wrapper

    return decorator


@beautiful_box(width_decor_symbol="~", height_decor_symbol="s")
def get_some_text(text):
    return text


print(get_some_text("Hello\nwoooooorld\nof\nmine!"))

+~~~~~~~~~~~~+
s Hello      s
s woooooorld s
s of         s
s mine!      s
+~~~~~~~~~~~~+


Еще декораторы можно накладывать друг на друга! Для этого нужно прописать вызов внешнего декоратора над вызовом внутреннего. Давайте добавим еще несколько коробок:

In [8]:
def beautiful_box(width_decor_symbol: str = "-", height_decor_symbol: str = "|"):
    def decorator(function):
        def wrapper(*args, **kwargs) -> str:
            text = function(*args, **kwargs)
            lines = text.split("\n")
            max_len = len(max(lines, key=len))
            border = f"+{'':{width_decor_symbol}>{max_len+2}}+"
            content = ""
            for line in lines:
                content += (
                    f"{height_decor_symbol} {line:<{max_len}} {height_decor_symbol}\n"
                )
            return f"{border}\n{content}{border}"

        return wrapper

    return decorator


@beautiful_box(width_decor_symbol=".", height_decor_symbol=":")
@beautiful_box(width_decor_symbol="~", height_decor_symbol="s")
@beautiful_box()
def get_some_text(text):
    return text


print(get_some_text("Hello\nwoooooorld\nof\nmine!"))

+....................+
: +~~~~~~~~~~~~~~~~+ :
: s +------------+ s :
: s | Hello      | s :
: s | woooooorld | s :
: s | of         | s :
: s | mine!      | s :
: s +------------+ s :
: +~~~~~~~~~~~~~~~~+ :
+....................+


Еще раз обратите внимание на порядок применения декораторов — самый нижний применится первым.

Декораторы нужны не только для того, чтобы рисовать коробки. Зачастую они применяются, чтобы скрыть сложную реализацию какого-то однотипного функционала. Мы уже знакомы с декоратором `@lru_cache`. Там реализован сложный механизм кеширования, скрытый за простым вызовом декоратора.

Другим примером использования декораторов являются серверные фреймворки. Например, **FastAPI** использует декораторы, чтобы интегрировать написанную функцию в рантайм сервера.

В файле `api_demo/main.py` лежит пример FastAPI-сервера. Тип эндпоинта и путь передаются при помощи декораторов.

In [9]:
!uvicorn demo.api.main:app --reload

INFO:     Will watch for changes in these directories: ['/Users/resdt/Desktop/lab6']
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [38567] using StatReload
INFO:     Started server process [38573]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
^C
INFO:     Finished server process [38573]
INFO:     Stopping reloader process [38567]


## Модули

Когда проект становится большим, его тоже разбивают на большие логические блоки и распределяют по файлам и папкам. Такие блоки называются модулями. Чтобы использовать функцию или переменную, которые указаны в модуле, его нужно импортировать. Делается это при помощи `import`, после которого через точку указывается путь к модулю:


In [10]:
import demo.modules.my_module1

demo.modules.my_module1.say_hello()

Hello


Путь получился очень длинным, и каждый раз прописывать его долго. Вместо этого можно пойти двумя путями:


1. Использовать алиас `as`:


In [11]:
import demo.modules.my_module1 as module1

module1.say_hello()

Hello


2. Использовать конструкцию `from module import function`:


In [12]:
from demo.modules.my_module1 import say_hello

say_hello()

Hello


Первый способ является более предпочтительным, так как исключает конфликт имен. В текущем модуле тоже может быть функция `say_hello`, и в этом случае она перезапишет имя импортируемой:


In [13]:
from demo.modules.my_module1 import say_hello


def say_hello():
    print("Goodbye hehe")


say_hello()

Goodbye hehe


Модули в файлах — это самостоятельные исполняемые скрипты. И код, который там прописан, выполняется при импорте. Но выполняться он будет только при первом импорте, так как модули кешируются и переиспользуются. Давайте убедимся в этом на примере:


In [14]:
import demo.modules.my_module2 as module2

module2.say_nothing()

Loaded module demo.modules.my_module2
Nothing


In [15]:
import demo.modules.my_module2 as module2

module2.say_nothing()

Nothing


На самом деле модули — не просто файлы, а полноценные объекты. У них тоже есть метаданные, которые их определяют. Такими данными являются:

- `__name__` — название модуля
- `__package__` — название пакета
- `__file__` — путь к файлу модуля
- `__spec__` — полная спецификация


In [16]:
import demo.modules.my_module2 as module2

print(module2.__name__)
print(module2.__package__)
print(module2.__file__)
print(module2.__spec__)

demo.modules.my_module2
demo.modules
/Users/resdt/Desktop/lab6/demo/modules/my_module2.py
ModuleSpec(name='demo.modules.my_module2', loader=<_frozen_importlib_external.SourceFileLoader object at 0x108afbfb0>, origin='/Users/resdt/Desktop/lab6/demo/modules/my_module2.py')


Модули бывают абсолютные и относительные. Абсолютный путь считается от точки запуска программы (это папка, в которой программа была запущена). Оносительный же путь начинается с точки и считается от файла, в котором он прописан. Например:

```python
import demo.modules.my_module2  # Абсолютный путь, пойдет от точки запуска программы
import .my_module2              # Будет искать `my_module2` в текущей папке
import ..my_module2             # Будет искать `my_module2` в папке на уровень выше
```


Принцип выбора относительного и абсолютного путей основывается на читаемости кода и его модульности. Если логика какой-то части приложения относительно независимая и не выходит за рамки одной-двух папок, то можно использовать относительный импорт. Если используются другие, более удаленные модули, то лучшим выбором станет абсолютный импорт.

Еще модуль, в котором есть относительный импорт, нельзя вызвать напрямую, нужно добавлять флаг `-m`. А также нельзя относительным импортом выйти за пределы корневого пакета. Таким образом, использовать их можно, но нужно помнить об ограничениях, которые они накладывают.
